In [1]:
import pandas as pd
from factue.utils.vars import PROJECT_ROOT
from factue.utils.viz import disp

In [2]:
sample_out = (
    PROJECT_ROOT
    / "data/02_cleaned_reference_test"
    / "dev"
    / "dev-eng"
    / "batch_0000.parquet"
)
df = pd.read_parquet(sample_out)
disp(df[["text", "reference", "check"]])

,text,reference,check
0,"The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More None",Photo shows Louis Armstrong as a child,"{ 'questions': [ {'id': 'meaning_preserved', 'answer': 'false', 'errors': ['The claim does not mention the Karnofsky family employing Louis Armstrong as a child, but rather that they employed a 7-year-old boy who later became a professional musician, which is likely referring to Louis Armstrong.']}, {'id': 'correct_named_entities', 'answer': 'true'}, {'id': 'correct_numbers', 'answer': 'false', 'errors': ['The claim mentions the child being shown as a photo, but does not mention any specific age or number related to the Karnofsky family employing him. However, it is mentioned that he was 7 years old when employed and 12 when no longer under their employ.']}, {'id': 'no_added_data', 'answer': 'false', 'errors': ['The claim mentions a photo of Louis Armstrong as a child, which is not present in the original text.']}, {'id': 'no_missing_data', 'answer': 'true'}, {'id': 'alternative_claim_formulation', 'answer': 'true', 'errors': ['A more accurate formulation would be: ""The Karnofsky Jewish family employed a 7-year-old boy who later became Louis Armstrong.""']} ] }"
1,"Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 mill Co This little guy was rescued after his mother was murdered by a hunter. His eyes say it all, don't you think? If you want to ban trophy hunting forever, spread this everywhere. Image Credit: FB. Changes: cropped, resized, text added. https://bit.ly/3b2vwas",This leopard cub's mother was killed by a trophy hunter,"{ 'questions': [ {'id': 'meaning_preserved', 'answer': 'true'}, {'id': 'correct_named_entities', 'answer': 'false', 'errors': ['""leopard cub"" is not present in the original text']}, {'id': 'correct_numbers', 'answer': 'true'}, {'id': 'no_added_data', 'answer': 'false', 'errors': ['""rescued"", ""murdered"", ""Image Credit: FB. Changes: cropped, resized, text added."" are not present in the original text']}, {'id': 'no_missing_data', 'answer': 'true'}, {'id': 'alternative_claim_formulation', 'answer': 'true'} ] }"
2,Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert None,Videos show current situation of Hyderabad amid heavy rain,"{ 'questions': [ {'id': 'meaning_preserved', 'answer': 'false', 'errors': ['The claim does not mention the heavy rain or crocodile alert, which are present in the original text.']}, {'id': 'correct_named_entities', 'a

In [ ]:
import json
from typing import Any, Dict


def safe_parse_json(val: Any) -> dict:
    if isinstance(val, dict):
        return val
    if isinstance(val, str):
        try:
            return json.loads(val)
        except json.JSONDecodeError:
            return {}
    return {}


def flatten_dict(
    d: Dict[Any, Any], parent_key: str = "", sep: str = "."
) -> Dict[str, Any]:
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for idx, item in enumerate(v):
                if isinstance(item, dict):
                    items.extend(
                        flatten_dict(item, f"{new_key}{sep}{idx}", sep=sep).items()
                    )
                else:
                    items.append((f"{new_key}{sep}{idx}", item))
        else:
            items.append((new_key, v))
    return dict(items)


def extract_check_fields(df: pd.DataFrame, check_column: str = "check") -> pd.DataFrame:
    # Safely parse and flatten 'check' column
    parsed_checks = df[check_column].apply(lambda x: flatten_dict(safe_parse_json(x)))

    # Find all keys that ever appear
    all_keys = set()
    for item in parsed_checks:
        all_keys.update(item.keys())

    # Create new columns
    for key in all_keys:
        df[key] = parsed_checks.apply(lambda x: x.get(key))

    return df


# Example usage
data = {
    "id": [1, 2, 3],
    "check": [
        '{"status": "ok", "value": 10, "meta": {"source": "sensor1"}, "questions": [{"id": 1, "text": "Q1"}, {"id": 2, "text": "Q2"}]}',
        {
            "status": "fail",
            "value": 5,
            "meta": {"source": "sensor2"},
            "questions": [{"id": 3, "text": "Q3"}],
        },
        '{\n"status": "ok",\n"value": 15,\n"meta": {"source": "sensor3"},\n"questions": []\n}',
    ],
}
extract_check_fields(df)